# NLP Ngram language model lab (unassessed)

In this lab you will do 4 exercises building and evaluating ngram language models of the following types:
1. A Maximum Likelihood Estimation (MLE) unigram model
2. A bigram model with add-one Laplace smoothing
3. A bigram model with general additive/Lidstone (add-k) smoothing
4. Ngram models with an advanced interpolation technique, Kneser-Ney snoothing (the methods are provided)

Before you start the exercises, make sure you run and understand the examples first. Then complete the exercises using the following 3 files with line-separated text to train the bigger language models on:
* training data -- "switchboard_lm_training.txt"
* heldout/development data -- "switchboard_lm_heldout.txt"
* test data -- "switchboard_lm_test.txt"

In [ ]:
from collections import Counter
from math import log
LOG_BASE = 2 # all the way through here we will use log base 2

In [ ]:
# Some useful methods for all exercises:
def glue_tokens(tokens, order):
    """A useful way of glueing tokens together for
    Kneser Ney smoothing and other smoothing methods

    :param: order is the order of the language model
        (1 = unigram, 2 = bigram, 3 =trigram etc.)
    """
    return '{0}@{1}'.format(order,' '.join(tokens))

def unglue_tokens(tokenstring, order):
    """Ungluing tokens glued by the glue_tokens method"""
    if order == 1:
        return [tokenstring.split("@")[1].replace(" ","")]
    return tokenstring.split("@")[1].split(" ")

def tokenize_sentence(sentence, order):
    """Returns a list of tokens with the correct numbers of initial
    and end tags (this is meant ot be used with a non-backoff model!!!)

    :sentence: a string of text
    :param: order is the order of the language model
        (1 = unigram, 2 = bigram, 3 =trigram etc.)
    """
    tokens = sentence.split()
    tokens = ['<s>'] * (order-1) + tokens + ['</s>']
    return tokens

In [ ]:
# see how glue/unglue tokens works:
glued = glue_tokens(["john"], 1)
unglued = unglue_tokens(glued, 1)
print(glued)
print(unglued)

1@john
['john']


# Examples

In [ ]:
# Set of sentences (corpus) from the example in the lecture slides
sentences = [
            "I am Sam",
            "Sam I am",
            "I do not like green eggs and ham"
            ]

## Example 1. Build a unigram MLE language model from a simple corpus.
An MLE unigram model will tell you how likely a word is to occur, estimated from the function of counts:

C(w_i)/N

where C(w_i) is the number of times the word at position i occurred in the training corpus, and N is the sum of the counts of all words, or, to put it another way, the length of the training corpus.

Notice the tokenization method adds a `</s>` at the end but no `<s>` is needed at the beginning of each sentence
    because unigrams do not have a context word (we are only concerned with the frequency of single words).

In [ ]:
unigrams = Counter()
for sent in sentences:
    words = tokenize_sentence(sent, 1)
    print("tokenized", words)
    for w in words:
        unigrams[w] +=1
unigram_total = sum(unigrams.values()) # to get the denominator for unigram probabilities
print(len(unigrams), "different unigrams observed")
print("unigram total", unigram_total)

tokenized ['I', 'am', 'Sam', '</s>']
tokenized ['Sam', 'I', 'am', '</s>']
tokenized ['I', 'do', 'not', 'like', 'green', 'eggs', 'and', 'ham', '</s>']
11 different unigrams observed
unigram total 17


In [ ]:
unigrams

Counter({'I': 3,
         'am': 2,
         'Sam': 2,
         '</s>': 3,
         'do': 1,
         'not': 1,
         'like': 1,
         'green': 1,
         'eggs': 1,
         'and': 1,
         'ham': 1})

To evaluate the model, we will measure the model's perplexity of those same training sentences. Note in normal practice you would want to do this on different data (as you will do below).

Perplexity is equal to 2 to the power of the cross entropy where cross entropy is the negative sum of all log probabilities from the model normalized by the length of the corpus N.

Measure the cross entropy and perplexity on each sentence too.

In [ ]:
s = 0  # total neg log prob mass for cross entropy
N = 0 # total number of words for normalizing s
for sent in sentences:
    # get the unigram model based probability of each sentence
    words = tokenize_sentence(sent, 1) # tokenize sentence with the order 1 as the parameter
    sent_s = 0  # recording non-normalized entropy for this sentence
    sent_N = 0  # total number of words in this sentence (for normalization)
    for w in words:
        prob = unigrams[w]/unigram_total
        logprob = log(prob, LOG_BASE)  # the log of the prob to base 2
        s += -log(prob, LOG_BASE) # add the neg log prob to s
        sent_s += -log(prob, LOG_BASE)  # add the neg log prob to sent_s
        N += 1 # increment the number of total words
        sent_N += 1 # increment the number of total words in this sentence
    sent_cross_entropy = sent_s/sent_N  # cross entropy total neg log probs/length for this sentence
    sent_perplexity = LOG_BASE ** sent_cross_entropy # perplexity for the sentence 2 to the cross-entropy
    print(words, "cross entropy:", sent_cross_entropy, "perplexity:", sent_perplexity)
cross_entropy = s/N # cross entropy of corpus total neg log probs/length
perplexity = 2 ** cross_entropy  # perplexity for the corpus 2 to the cross-entropy
print("unigram corpus cross entropy", cross_entropy)
print("unigram corpus perplexity", perplexity)

['I', 'am', 'Sam', '</s>'] cross entropy: 2.7949815908897615 perplexity: 6.940220937885672
['Sam', 'I', 'am', '</s>'] cross entropy: 2.7949815908897615 perplexity: 6.940220937885672
['I', 'do', 'not', 'like', 'green', 'eggs', 'and', 'ham', '</s>'] cross entropy: 3.7352489522011934 perplexity: 13.317477627933627
unigram corpus cross entropy 3.2927701939369913
unigram corpus perplexity 9.799921507045037


## Example 2. Build a bigram MLE language model from the same corpus
A MLE (unsmoothed) bigram model will tell you how likely a word is to occur given the previous word, estimated from the function of counts:

C(w_i-1, w_i)/C(w_i-1)

where for any pairs of contiguous words w_i-1, w_i, C(w_i-1, w_i) is the number of times the word at position i follows the word at position i-1 in the training corpus, and C(w_i-1) is the number of times the word at position i-1 occurs in the corpus. E.g. for the bigram probability of 'john likes', C(w_i-1, w_i) is the number of times 'john likes' occurs, and C(w_i-1) is how many times 'john' occurs.

Notice the tokenization method adds a `</s>` at the end and also one `<s>` for padding at the beginning as we want to count the number of times the word at the beginning of each sentence begins a sentence.

In [ ]:
# First get the counts from the training corpus for bigrams without smoothing
bigrams = Counter() # a counter for how many times a given bigram sequence w_i-1,w_i occurs
bigram_context = Counter() # a counter for how many times each word is used as a context word w_i-1 (so will include the start symbol)
order = 2 # order (i.e. n) of the language model- bigram n=2
for s in sentences:
    words = tokenize_sentence(s, order)  # tokenize sentence with the order 2 as the parameter
    print(words)
    for i in range(order - 1, len(words)):
        context = words[i-order+1:i]
        target = words[i]
        ngram = context + [target]
        bigrams[glue_tokens(ngram, order)] +=1
        bigram_context[glue_tokens(context, 1)] += 1
print(len(bigrams.keys()), "different bigrams")
print(len(bigram_context.keys()), "different bigram contexts (and unigrams) observed")

['<s>', 'I', 'am', 'Sam', '</s>']
['<s>', 'Sam', 'I', 'am', '</s>']
['<s>', 'I', 'do', 'not', 'like', 'green', 'eggs', 'and', 'ham', '</s>']
15 different bigrams
11 different bigram contexts (and unigrams) observed


In [ ]:
bigrams

Counter({'2@<s> I': 2,
         '2@I am': 2,
         '2@am Sam': 1,
         '2@Sam </s>': 1,
         '2@<s> Sam': 1,
         '2@Sam I': 1,
         '2@am </s>': 1,
         '2@I do': 1,
         '2@do not': 1,
         '2@not like': 1,
         '2@like green': 1,
         '2@green eggs': 1,
         '2@eggs and': 1,
         '2@and ham': 1,
         '2@ham </s>': 1})

In [ ]:
bigram_context

Counter({'1@<s>': 3,
         '1@I': 3,
         '1@am': 2,
         '1@Sam': 2,
         '1@do': 1,
         '1@not': 1,
         '1@like': 1,
         '1@green': 1,
         '1@eggs': 1,
         '1@and': 1,
         '1@ham': 1})

In [ ]:
# a handy function to calculate the probability of an bigram from the counts
def prob_bigram_MLE(ngram):
    """A simple function to compute the
    MLE probability estimation based on the counts.
    Follows the equation:
    C(w_i-1, w_i)/C(w_i-1)

    Dictionaries bigrams and bigram_context are global variables.

    """
    numerator = bigrams[glue_tokens(ngram, 2)]
    denominator = bigram_context[glue_tokens(ngram[:1], 1)]
    prob = numerator / denominator
    return prob


if False: # optional- see if the continuation probabilities sum to 1
    # check if each bigram continuation distribution sums to one
    # look at the distributions of possible continuations after each word
    for context, v in bigram_context.items():
        context = unglue_tokens(context, 1)
        print("%% context", context)
        check_ngram_total_sums_to_1 = 0
        # for a given context the continuation probabilities
        # over the whole vocab should sum to 1
        for u in unigrams.keys():
            ngram = context + [u]
            prob = prob_bigram_MLE(ngram)
            print(ngram, prob)
            check_ngram_total_sums_to_1 += prob
        print("sums to 1?", check_ngram_total_sums_to_1)

In [ ]:
bigrams

Counter({'2@<s> I': 2,
         '2@<s> Sam': 1,
         '2@I am': 2,
         '2@I do': 1,
         '2@Sam </s>': 1,
         '2@Sam I': 1,
         '2@am </s>': 1,
         '2@am Sam': 1,
         '2@and ham': 1,
         '2@do not': 1,
         '2@eggs and': 1,
         '2@green eggs': 1,
         '2@ham </s>': 1,
         '2@like green': 1,
         '2@not like': 1})

In [ ]:
# Check the estimates for the lecture examples:
# p(I|<s>)
# p(Sam|<s>)
# p(am|I)
# p(</s>|Sam)
# p(Sam|am)
# p(do|I)

print(prob_bigram_MLE(['<s>','I']))
print(prob_bigram_MLE(['<s>', 'Sam']))
print(prob_bigram_MLE(['I', 'am']))
print(prob_bigram_MLE(['Sam', '</s>']))
print(prob_bigram_MLE(['am', 'Sam']))
print(prob_bigram_MLE(['I', 'do']))

0.6666666666666666
0.3333333333333333
0.6666666666666666
0.5
0.5
0.3333333333333333


To evaluate the model, as in the unigram case above we will measure the model's perplexity of those same training sentences.

Notice that even with this small corpus the bigram perplexity is significantly lower than the unigram perplexity.

In [ ]:
s = 0  # total neg log prob mass for cross entropy
N = 0 # total number of words for normalizing s
for sent in sentences:
    words = tokenize_sentence(sent, order)
    sent_s = 0  # recording non-normalized entropy for this sentence
    sent_N = 0  # total number of words in this sentence (for normalization)
    for i in range(order - 1, len(words)):
        context = words[i-order+1:i]
        target = words[i]
        ngram = context + [target]
        prob = prob_bigram_MLE(ngram)
        s += -log(prob, LOG_BASE) # add the neg log prob to s
        sent_s += -log(prob, LOG_BASE)  # add the neg log prob to sent_s
        N += 1 # increment the number of total words
        sent_N += 1 # increment the number of total words in this sentence
    sent_cross_entropy = sent_s/sent_N  # cross entropy total neg log probs/length for this sentence
    sent_perplexity = LOG_BASE ** sent_cross_entropy # perplexity for the sentence 2 to the cross-entropy
    print(words, "cross entropy:", sent_cross_entropy, "perplexity:", sent_perplexity)
cross_entropy = s/N # cross entropy of corpus total neg log probs/length
perplexity = 2 ** cross_entropy  # perplexity for the corpus 2 to the cross-entropy
print("bigram corpus cross entropy", cross_entropy)
print("bigram corpus perplexity", perplexity)

NameError: ignored

# Exercises

# Exercise 1. Defining Vocabulary, using OOV word token and building a unigram MLE model from a bigger corpus

Write code to read in the file `switchboard_lm_train.txt` which has preprocessed text on each line.

You will populate a unigram language model based on that data for an MLE estimation using a `Counter` using this data (see Example 1 above as to how this is done for a smaller dataset).

Before you do this, you have to define a vocabulary of words using the training data, which you will keep the same for all of the following exercises. In these exercises, the vocabulary is defined by using a Minimum Document Frequency of 2 in the training data. That means the vocab should only contain words which occur at least twice in the training data.

Any words not in the vocabulary in the training, heldout and testing data must be replaced with an out-of-vocab symbol `<unk/>` before processing them.

Using this model, calculate the perplexity of the ENTIRE test corpus `switchboard_lm_test.txt`- again, remember to replace words unknown by the model with `<unk/>` before getting these measures. See Example 1 as to how this is done for a unigram model on the smaller example dataset.


# Exercise 2. Bigram model with add-one smoothing

Change your method for reading in and training a language model from Exercise 1 so it works for bigrams. Use the same methods for identifying and replacing out-of-vocabulary words as you did in Exercise 1 (i.e. use the same vocabulary).

However, in testing, rather than just using the raw counts for implementing MLE probabilities you should implement add-one smoothing (see the lecture notes and Jurafsky & Martin Chapter 3/Manning and Schuetze Chapter 6). Remember this involves using the vocabulary size in the denominator of the formula.

Obtain the perplexity score on the test data as above for this smoothed bigram model.

# Exercise 3. Bigram model with general additive (Lidstone) add-k smoothing

Modify your code from Exercise 2 such that it generalizes beyond adding 1 to all counts, but can add differing amounts k of mass instead, to implement general additive add-k smoothing.

On the HELDOUT corpus `switchboard_lm_heldout.txt` experiment with different values of k (e.g. 0.2, 0.4, 0.6, 0.8, though try others if you can) and report the perplexity scores for these different values in a comment. You could also use an optimization algorithm from scipy.optimize to search this efficiently.

Once you find the value which gives you the lowest perplexity on the heldout data, use this model to get the perplexity of the test data once and report the scores in a comment. You should be able to get better (lower) perplexity scores than plus-1 smoothing, however make sure the vocabulary used is the same, else it is not a fair comparison.


# Exercise 4. Ngram models with Kneser-Ney smoothing

Kneser-Ney smoothing is a state-of-the-art technique for smoothing n-gram models.

The algorithm is quite complicated, and is implemented for you below for training/getting the appropriate counts  on the training data (`count_ngrams_interpolated_kneser_ney()`). The training process is implemented below for a trigram model, but without doing the appropriate out-of-vocab word replacement as you've done above, using exactly the same vocabulary.

The application at test time is done with the method `kneser_ney_ngram_prob()` using the trained Counters, which gives the probability of the model applied to an ngram of a given order, with a given discount.

Try to follow how the training works and how the application of the model to ngrams works, and refer to both Section 3.5 in Jurafsky and Martin and the below article on QM plus (pages 7-8 particularly):

"A Bit of Progress in Language Modeling" (2001) - Joshua T. Goodman

In this exercise, you will first modify the training part of the code so it does the replacement of out-of-vocab words as you did in the previous exercises. You do not need to modify the methods below.

On the HELDOUT corpus experiment with different orders from trigram upwards (try 3, 4 and 5) and different discount values (e.g. 0.2, 0.4, 0.6, 0.8, though try others if you can) and report the perplexity scores for these different values in a comment. You could also use an optimization algorithm from scipy.optimize to search the different n values and discount values efficiently.

Once you find the order and discount values which gives you the lowest perplexity on the heldout data, use this model to get the perplexity of the TEST data once and report the scores in a comment. You should be able to beat (get a lower) perplexity for this compared to excercises 1-3, though make sure you are always using the same vocabulary, else it is not a fair comparison.

In [ ]:
# Kneser-Ney smoothing
order = 3  # use a trigram
discount = 0.8  # use a discount of 0.8


# initialize the maps and counts
unigram_denominator = 0
ngram_numerator_map = Counter()
ngram_denominator_map = Counter()
ngram_non_zero_map = Counter()


def count_ngrams_interpolated_kneser_ney(tokens,
                                   order,
                                   ngram_numerator_map,
                                   ngram_denominator_map,
                                   ngram_non_zero_map,
                                   unigram_denominator):
    """Function used in n-gram language model training
    to count the n-grams in tokens and also record the
    lower order non -ero counts necessary for interpolated Kneser-Ney
    smoothing.

    Taken from Goodman 2001 and generalized to arbitrary orders"""
    for i in range(order-1,len(tokens)): # tokens should have a prefix of order - 1
        #print(i)
        for d in range(order,0,-1): #go through all the different 'n's backwards
            if d == 1:
                unigram_denominator += 1   # it has been in a context
                ngram_numerator_map[glue_tokens(tokens[i],d)] += 1
            else:
                den_key = glue_tokens(tokens[i-(d-1) : i], d)
                num_key = glue_tokens(tokens[i-(d-1) : i+1], d)
                # increment the number of times the denominator/context has been seen by 1
                ngram_denominator_map[den_key] += 1
                # we store the numerator value to check if it's 0
                tmp = ngram_numerator_map[num_key]
                # we increment the number of times it's been observed as a numerator
                ngram_numerator_map[num_key] += 1
                # this incrementation of the n-gram count if d < order
                # will only happen if the ngram tested at d+1 was unique
                if tmp == 0:
                    # if this is the first time we see this ngram
                    # increment number of types for which its context
                    # has been used as a context for any continuation
                    ngram_non_zero_map[den_key] += 1
                else:
                    break
                    # if the ngram has already been seen
                    # we don't go down to any lower order models
                    # because they will have already been counted as types
    # return the updated counts and maps
    return ngram_numerator_map, ngram_denominator_map, ngram_non_zero_map, unigram_denominator


In [ ]:
# train the model by populating the maps and counts
corpus = open("switchboard_lm_train.txt")
for line in corpus:
    tokens = tokenize_sentence(line, order)
    ngram_numerator_map, ngram_denominator_map, ngram_non_zero_map, unigram_denominator =\
            count_ngrams_interpolated_kneser_ney(tokens,
                                           order,
                                           ngram_numerator_map,
                                           ngram_denominator_map,
                                           ngram_non_zero_map,
                                           unigram_denominator)
corpus.close()

In [ ]:
def kneser_ney_ngram_prob(ngram, discount, order):
    """KN smoothed ngram probability from Goodman 2001.
    This is run at test time to calculate the probability
    of a given n-gram or a given order with a given discount.

    ngram :: list of strings, the ngram
    discount :: float, the discount used
    order :: int, order of the model
    """
    # First, calculate the unigram continuation prob of the last token
    # If we've never seen it at all, it will
    # have no probability as a numerator
    uni_num = ngram_numerator_map.get(glue_tokens(ngram[-1], 1)) # number of bigrams the word is a continuation for
    if not uni_num: # if no value found in dict, make it 0
        uni_num = 0
    # unigram_denominator is the number of different bigram types (not tokens)
    probability = previous_prob = uni_num / unigram_denominator

    # Check: Given <unk/> should have been used in place of unknown words before passing
    # to this method, probability should be non-zero
    if probability == 0.0:
        print("0 prob for unigram!")
        print(glue_tokens(ngram[-1], 1))
        print(ngram)
        print(ngram_numerator_map.get(glue_tokens(ngram[-1], 1)))
        print(unigram_denominator)
        raise Exception

    # Compute the higher order probs (from 2/bi-gram upwards) and interpolate them
    for d in range(2,order+1):
        # Get the context count for the denominator:
        # When d = order (n) this is the number of times it's observed in the corpus (tokens)
        # When d < order (n) this is the number of different continuation types (not tokens) seen with it as its prefix
        ngram_denom = ngram_denominator_map.get(glue_tokens(ngram[-(d):-1], d))
        if not ngram_denom: # if no value found in dict, make it 0
            ngram_denom = 0
        if ngram_denom != 0:
            # Get the ngram count for the numerator
            # When d = order (n) this is the number of times it's observed in the corpus (tokens)
            # When d < order (n) this is the number of types of unique ngram for n=d+1 it is a continuation for (types)
            ngram_num = ngram_numerator_map.get(glue_tokens(ngram[-(d):], d))
            if not ngram_num: # if no value found in dict, make it 0
                ngram_num = 0
            if ngram_num != 0:
                # calculate the prob with fixed discount
                current_prob = (ngram_num - discount) / ngram_denom
            else:
                current_prob = 0.0
            # get the number of word types that can follow the context
            # (number of times normalised discount has been applied):
            nonzero = ngram_non_zero_map.get(glue_tokens(ngram[-(d):-1], d))
            if not nonzero: # if no value found in dict, make it 0
                nonzero = 0
            # get the lambda for this context
            lambda_context = discount / ngram_denom * nonzero
            # interpolate with previous probability of lower orders calculated so far
            current_prob += lambda_context * previous_prob
            previous_prob = current_prob
            probability = current_prob
        else:
            #if this context (e.g. bigram context for trigrams) has never been seen,
            #then we can only use the last order with a probability (e.g. unigram)
            #and halt
            probability = previous_prob
            break
    return probability

In [ ]:
kneser_ney_ngram_prob(ngram, discount, order)

ZeroDivisionError: ignored